![sql](img/sql-logo.jpg)

# A guide to databases, SQL, & using them with `pandas`

Let's start with Peter Bell's great blog piece [on relational databases](https://flatironschool.com/blog/an-introduction-to-the-relational-database).

Themes:
- Structured vs. Unstructured Data
- Records (Rows) in Tables in Databases
- Eliminating Redundancy

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Group Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

This sort of picture is sometimes called an **E**ntity **R**elationship **D**iagram. [Here](https://www.smartdraw.com/entity-relationship-diagram/) is another good resource for explaining typical ERD conventions.

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute(
"""
SELECT name
FROM sqlite_master
WHERE type='table';
"""
)
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


For more on "sqlite_master", see [here](https://www.techonthenet.com/sqlite/sys_tables/index.php).

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [4]:
cursor.execute(
"""
SELECT *
FROM airports
"""
)

#### Use description

In [5]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

The septuple structure has to do with Python API [compatibility issues](https://kite.com/python/docs/sqlite3.Cursor.description).

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`. The latter takes a number of records to return as a parameter.

In [6]:
cursor.fetchmany(10)

[(0,
  '1',
  'Goroka',
  'Goroka',
  'Papua New Guinea',
  'GKA',
  'AYGA',
  '-6.081689',
  '145.391881',
  '5282',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (1,
  '2',
  'Madang',
  'Madang',
  'Papua New Guinea',
  'MAG',
  'AYMD',
  '-5.207083',
  '145.7887',
  '20',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (6,
  '7'

**Task:** Get the descriptive data for airlines and routes tables

In [7]:
cursor.execute(
"""
SELECT *
FROM airlines
"""
)

cursor.fetchall()

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N'),
 (5,
  '6',
  '223 Flight Unit State Airline',
  '\\N',
  None,
  'CHD',
  'CHKALOVSK-AVIA',
  'Russia',
  'N'),
 (6,
  '7',
  '224th Flight Unit',
  '\\N',
  None,
  'TTF',
  'CARGO UNIT',
  'Russia',
  'N'),
 (7,
  '8',
  '247 Jet Ltd',
  '\\N',
  None,
  'TWF',
  'CLOUD RUNNER',
  'United Kingdom',
  'N'),
 (8, '9', '3D Aviation', '\\N', None, 'SEC', 'SECUREX', 'United States', 'N'),
 (9,
  '10',
  '40-Mile Air',
  '\\N',
  'Q5',
  'MLA',
  'MILE-AIR',
  'United States',
  'Y'),
 (10, '11', '4D Air', '\\N', None, 'QRT', 'QUARTET', 'Thailand', 'N'),
 (11,
  '12',
  '611897 

In [8]:
cursor.execute(
"""
SELECT *
FROM routes
"""
)

cursor.fetchall()

[(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2'),
 (1, '2B', '410', 'ASF', '2966', 'KZN', '2990', None, '0', 'CR2'),
 (2, '2B', '410', 'ASF', '2966', 'MRV', '2962', None, '0', 'CR2'),
 (3, '2B', '410', 'CEK', '2968', 'KZN', '2990', None, '0', 'CR2'),
 (4, '2B', '410', 'CEK', '2968', 'OVB', '4078', None, '0', 'CR2'),
 (5, '2B', '410', 'DME', '4029', 'KZN', '2990', None, '0', 'CR2'),
 (6, '2B', '410', 'DME', '4029', 'NBC', '6969', None, '0', 'CR2'),
 (7, '2B', '410', 'DME', '4029', 'TGK', '\\N', None, '0', 'CR2'),
 (8, '2B', '410', 'DME', '4029', 'UUA', '6160', None, '0', 'CR2'),
 (9, '2B', '410', 'EGO', '6156', 'KGD', '2952', None, '0', 'CR2'),
 (10, '2B', '410', 'EGO', '6156', 'KZN', '2990', None, '0', 'CR2'),
 (11, '2B', '410', 'GYD', '2922', 'NBC', '6969', None, '0', 'CR2'),
 (12, '2B', '410', 'KGD', '2952', 'EGO', '6156', None, '0', 'CR2'),
 (13, '2B', '410', 'KZN', '2990', 'AER', '2965', None, '0', 'CR2'),
 (14, '2B', '410', 'KZN', '2990', 'ASF', '2966', None, '0',

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**SELECT**

**FROM**

**WHERE**

**GROUP BY**

**ORDER BY**

**LIMIT**

#### Options for each:

**Select**:  `DISTINCT`, using `AS` to rename columns, single number aggregates, `COUNT()` to count, `*` for "all"

**From:** also uses aliasing with `AS`

**Where**: `=`, `BETWEEN`, `IN`, wildcards with `%`, `AND`, `OR`, `NOT` pattern matching with `LIKE` and `ILIKE`

**Order by**: `ASC` and `DESC`

**Limit**:  #

***

**Task**: 
- Select only active airlines in the United Kingdom from the airlines table <br/>
Hint: The values for 'active' are 'Y' and 'N'.
- Select the unique list of countries with airports

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table. <br/>
Hint: Which column can serve as the link between the two tables? That is, which column in `airports` can serve as foreign key to `routes`?

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest numbers of active airlines?
- Which countries have the highest numbers of inactive airlines?
- What about airports by time zone?

In [9]:
cursor.execute(
"""
SELECT COUNT() num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30
"""
).fetchall()

[(141, 'United States'),
 (72, 'Russia'),
 (40, 'United Kingdom'),
 (37, 'Germany'),
 (34, 'Canada'),
 (26, 'Australia'),
 (25, 'China'),
 (24, 'Spain'),
 (23, 'Brazil'),
 (22, 'France'),
 (19, 'Japan'),
 (18, 'Italy'),
 (17, 'India'),
 (17, 'Indonesia'),
 (16, 'Thailand'),
 (16, 'Turkey'),
 (15, 'Sweden'),
 (14, 'Switzerland'),
 (13, 'Portugal'),
 (13, 'Ukraine'),
 (12, 'Austria'),
 (12, 'Egypt'),
 (12, 'Finland'),
 (12, 'Mexico'),
 (11, 'Peru')]

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [10]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/\
views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [11]:
shelter_data[shelter_data['Name'] == "Max"].head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
161,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
380,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
503,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
811,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


In [12]:
# With .query()

max_data = shelter_data.query('Name == "Max"')
max_data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
161,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
380,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
503,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
811,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


## Goal 6: Transfering from sqlite to pandas

`.read_sql_query()`

In [13]:
import pandas as pd

conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("SELECT * FROM airports LIMIT 50", conn)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
5,5,6,Wewak Intl,Wewak,Papua New Guinea,WWK,AYWK,-3.583828,143.669186,19,10,U,Pacific/Port_Moresby
6,6,7,Narsarsuaq,Narssarssuaq,Greenland,UAK,BGBW,61.160517,-45.425978,112,-3,E,America/Godthab
7,7,8,Nuuk,Godthaab,Greenland,GOH,BGGH,64.190922,-51.678064,283,-3,E,America/Godthab
8,8,9,Sondre Stromfjord,Sondrestrom,Greenland,SFJ,BGSF,67.016969,-50.689325,165,-3,E,America/Godthab
9,9,10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531203,-68.703161,251,-4,E,America/Thule


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## On Your Own

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
# Getting Started

con = sqlite3.connect('Chinook_Sqlite.sqlite')

In [ ]:
cursor2 = con.cursor()

In [ ]:
cursor2.execute(
"""
SELECT tbl_name
FROM sqlite_master
WHERE type='table'
"""
)

In [ ]:
cursor2.fetchall()